In [2]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..")))
import packages.Preprocesamiento as ppr
import os
import datetime as dt
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import TweedieRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_validate, GridSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score

In [3]:
path_datos = os.path.join('..','..','Datos','Limpios')
filename = os.path.join(path_datos,'datos2_limpios.csv')
df2 = pd.read_csv(filename, index_col= 0)
df2.head()

,Edad,Ingreso Anual (€),Porcentaje Ahorro (%),Ahorro Actual (€),Gasto Mensual (€),Vida Esperada (años),Edad de Jubilación,Años Hasta Jubilación
0,32,29353.06,7.26,25554.94,2268.62,81.29,67,35
1,51,39218.38,23.72,288438.59,2492.83,75.00,67,16
2,32,15000.00,11.14,20055.00,1110.73,78.70,65,33
3,30,23517.91,25.01,58819.72,1469.66,75.00,67,37
4,60,20867.64,23.96,200005.33,1322.29,81.47,62,2


In [4]:
df2_limpio = df2.drop(['Porcentaje Ahorro (%)','Ahorro Actual (€)', 'Gasto Mensual (€)', 'Vida Esperada (años)',  'Edad de Jubilación'], axis=1)
df2_limpio.describe()

,Edad,Ingreso Anual (€),Años Hasta Jubilación
count,5000.000000,5000.000000,5000.000000
mean,42.392600,25851.308952,22.086000
std,12.622659,12820.844484,12.722774
min,21.000000,15000.000000,0.000000
25%,32.000000,15617.382500,11.000000
50%,43.000000,22069.875000,22.000000
75%,53.000000,31068.317500,33.000000
max,64.000000,100000.000000,46.000000


In [5]:
X=df2_limpio.iloc[:,:2]
y=df2_limpio.iloc[:,-1]

In [6]:
X

,Edad,Ingreso Anual (€)
0,32,29353.06
1,51,39218.38
2,32,15000.00
3,30,23517.91
4,60,20867.64
...,...,...
4995,58,23759.51
4996,46,15000.00
4997,51,23620.10
4998,61,15000.00


In [7]:
models=[LinearRegression(),Lasso(max_iter=1000),Ridge(max_iter=1000),ElasticNet(max_iter=1000),TweedieRegressor(max_iter=10000)]

for model in models:
    cv_score=cross_validate(model,X,y,scoring=['neg_mean_absolute_error','r2'],cv=5)
    
    mean_mae = sum(cv_score['test_neg_mean_absolute_error'])/len(cv_score['test_neg_mean_absolute_error'])
    mean_r2 = sum(cv_score['test_r2'])/len(cv_score['test_r2'])
    print("MAE for model",model,'=',mean_mae)
    print("R2 for model",model,'=',mean_r2)

MAE for model LinearRegression() = -1.4370400554484437
R2 for model LinearRegression() = 0.9820220734365215
MAE for model Lasso() = -1.4367098628540347
R2 for model Lasso() = 0.9819838151704363
MAE for model Ridge(max_iter=1000) = -1.43703997229671
R2 for model Ridge(max_iter=1000) = 0.9820220736104247
MAE for model ElasticNet() = -1.436709682523483
R2 for model ElasticNet() = 0.9819841478086324
MAE for model TweedieRegressor(max_iter=10000) = -1.4367096477343115
R2 for model TweedieRegressor(max_iter=10000) = 0.9819844781322381


In [8]:
model_hyperparameters = {
    'lin_reg':{
      
    },
    'lasso_reg':{
        'alpha':[0.5,1,5,10,20]        
    },
    'ridge_reg':{
        'alpha':[0.5,1,5,10,20]        
    },
    'elastic_reg':{
        'alpha':[0.5,1,5,10,20],
        'l1_ratio':[0.1,0.3,0.5,0.7,0.9]        
    },
    'tweedie_reg':{
        'power':[0,1,2,3],
        'alpha':[0, 0.5, 1, 5, 10, 20],    
        'link':['log','identity']    
    }
}

In [9]:
model_keys=list(model_hyperparameters.keys())

In [10]:
result = []
i=0

for model in models:
    key=model_keys[i]
    i+=1
    params = model_hyperparameters[key]

    regressor=GridSearchCV(model,params,cv=5,scoring=['neg_mean_absolute_error','r2'],refit='neg_mean_absolute_error')

    regressor.fit(X,y)
    result.append({
        'model_used':model,
        'highest_score':regressor.best_score_,
        'best hyperparameters':regressor.best_params_
    })

c:\bdata2\reto07\PROYECTO_PYHTON\.venv\Lib\site-packages\sklearn\linear_model\_glm\glm.py:285: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
c:\bdata2\reto07\PROYECTO_PYHTON\.venv\Lib\site-packages\sklearn\linear_model\_glm\glm.py:285: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
c:\bdata2\reto07\PROYECTO_PYHTON\.venv\Lib\site-packages\sklearn\linear_model\_glm\glm.py:285: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable

In [11]:
pd.DataFrame(result)

,model_used,highest_score,best hyperparameters
0,LinearRegression(),-1.437040,{}
1,Lasso(),-1.436710,{'alpha': 1}
2,Ridge(max_iter=1000),-1.437038,{'alpha': 20}
3,ElasticNet(),-1.436710,"{'alpha': 1, 'l1_ratio': 0.1}"
4,TweedieRegressor(max_iter=10000),-1.436710,"{'alpha': 1, 'link': 'identity', 'power': 0}"


In [12]:
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model=TweedieRegressor(max_iter=10000,alpha=1,link='identity',power=0)
model.fit(X_train,y_train)
predictions=model.predict(X_test)
score = mean_absolute_error(y_test,predictions)
print("MAE for model",model,'=', score)

MAE for model TweedieRegressor(alpha=1, link='identity', max_iter=10000, power=0) = 1.4559236525033241


In [13]:
predictions

array([23.45810313, 22.47902414,  3.63057638, ..., 13.54424382,
       17.49404471, 12.55401732], shape=(1500,))

In [14]:
print("Train RMSE: ", root_mean_squared_error(y_train, model.predict(X_train)))
print("Train R2: ", r2_score(y_train, model.predict(X_train)))

print("Test RMSE: ", root_mean_squared_error(y_test, model.predict(X_test)))
print("Test R2: ", r2_score(y_test, model.predict(X_test)))

Train RMSE:  1.693317770213319
Train R2:  0.9823338625546076
Test RMSE:  1.7321479640447315
Test R2:  0.9813308333845465


In [15]:
path_datos = os.path.join('..','..','Datos','Limpios')
filename = os.path.join(path_datos,'datos1_limpios.csv')
df = pd.read_csv(filename, index_col= 0)
df.head()

,ID,FECHA NAC,SEXO,NOMINA BRUTA 01/01/2025,FECHA ENTRADA,PARA CONTAR MESES,EDAD
0,1,1974-03-22,H,15319.07,2002-05-01,2002-05-01,51
1,2,1995-06-21,M,19168.80,2018-04-13,2018-05-01,30
2,3,1973-06-07,H,18719.86,2002-04-01,2002-04-01,52
3,4,1975-07-03,M,12942.60,2002-04-01,2002-04-01,50
4,5,1976-07-19,M,20158.08,2003-04-01,2003-04-01,49


In [16]:
X_2 = df.drop(['ID', 'FECHA NAC', 'SEXO', 'FECHA ENTRADA', 'PARA CONTAR MESES'], axis=1)

In [17]:
X_2.columns = ['Ingreso Anual (€)','Edad']
X_2 = X_2[['Edad','Ingreso Anual (€)']] 
X_2

,Edad,Ingreso Anual (€)
0,51,15319.07
1,30,19168.80
2,52,18719.86
3,50,12942.60
4,49,20158.08
...,...,...
95,50,25222.69
96,34,18759.95
97,26,22175.25
98,36,25972.71


In [18]:
predicciones=model.predict(X_2)

In [21]:
df['AÑOS HASTA JUBILACION'] = predicciones

In [23]:
df['EDAD DE JUBILACION'] = df['EDAD'] + df['AÑOS HASTA JUBILACION']

In [24]:
df.head()

,ID,FECHA NAC,SEXO,NOMINA BRUTA 01/01/2025,FECHA ENTRADA,PARA CONTAR MESES,EDAD,AÑOS HASTA JUBILACION,EDAD DE JUBILACION
0,1,1974-03-22,H,15319.07,2002-05-01,2002-05-01,51,13.554036,64.554036
1,2,1995-06-21,M,19168.80,2018-04-13,2018-05-01,30,34.378970,64.378970
2,3,1973-06-07,H,18719.86,2002-04-01,2002-04-01,52,12.558925,64.558925
3,4,1975-07-03,M,12942.60,2002-04-01,2002-04-01,50,14.548162,64.548162
4,5,1976-07-19,M,20158.08,2003-04-01,2003-04-01,49,15.533061,64.533061


In [20]:
predicciones

array([13.55403595, 34.3789702 , 12.55892499, 14.54816166, 15.53306122,
       25.45896275, 32.39308451, 17.51913231, 10.57887235, 21.48102598,
       39.33136206, 10.56858623, 21.47728906, 19.50270865, 21.48671474,
       26.4365812 , 31.40567001, 39.3396993 , 36.36818234, 28.41952433,
       21.47675966, 15.53561127, 19.49388834, 30.41173416, 15.53249223,
       36.36460621, 28.43401962, 17.51490516, 30.41432828, 32.40194384,
       17.51538239, 34.37171524, 22.48294422, 35.37209373, 26.43703125,
       29.41775211, 24.4610853 , 38.34954332, 28.42932206, 28.41959714,
       34.37696707, 27.44250002, 32.39961319, 27.43163518, 21.48293354,
       20.49478107, 12.55048481, 10.56736346, 33.37582879, 36.35207511,
       34.3758278 , 30.40601203, 31.40039463, 30.41052095, 37.34784036,
       36.35533019, 18.5084351 , 37.35398232, 26.44186811, 14.54224064,
       13.54812398, 19.4916556 , 39.32866395, 38.34653722, 24.45915324,
       30.41185323, 14.5403724 , 38.33773021, 38.34615303,  8.57